# 从海量文本中提取主题与关键词

* 非监督学习(unsupervised machine learning)
* 具体而言，我们需要用到的方法叫主题建模(topic model)或者主题抽取(topic extraction)。
* 主题模型：在机器学习和自然语言处理等领域是用来在一系列文档中发现抽象主题的一种统计模型。
* 主题抽取有若干方法。目前最为流行的叫做隐含狄利克雷分布(Latent Dirichlet allocation)，简称LDA。

In [1]:
import pandas as pd

In [2]:
# 读入datascience.csv ,注意他的编码是中文GB18030，不是Pandas的默认编码，所以此处需要指定编码类型，避免乱码。
# __怎样产看 .csv 文件的编码类型的问题未解决。。__
df = pd.read_csv('datascience.csv', encoding='gb18030')

In [3]:
df.head()

,title,author,content
0,大数据产业迎政策暖风 最新大数据概念股一览,财经热点扒客,大数据产业发展受到国家重视，而大数据已经上升为国家战略，未来发展前景很广阔。大数据产业“十三...
1,Google发布机器学习平台Tensorflow游乐场～带你一起玩神经网络！,硅谷周边,点击上方“硅谷周边”关注我，收到最新的文章哦！昨天，Google发布了Tensorflow游...
2,李克强：中国大数据和云计算产业是开放的,苏州高新区金融办,国务院总理李克强当地时间20日上午在纽约下榻饭店同美国经济、金融、智库、媒体等各界人士座谈，...
3,全峰集团持续挖掘大数据,快递物流网,2016年，全峰集团持续挖掘大数据、云计算、“互联网+”等前沿技术和物流快递的融合，并通过优...
4,第366期【微理工】贵州理工学院召开大数据分析与应用专题分享会,贵州理工学院,贵州理工学院召开大数据分析与应用专题分享会 借“创响中国”贵安站巡回接力活动暨2016贵安大...


In [4]:
df.shape

(1024, 3)

In [5]:
import jieba

In [6]:
def chinese_word_cut(mytext):
    return ' '.join(jieba.cut(mytext))

In [7]:
df['content_cutted'] = df.content.apply(chinese_word_cut)
# 下面这段提示没有特别的影响

Building prefix dict from the default dictionary ...
Loading model from cache c:\users\21pirlo\appdata\local\temp\jieba.cache
Loading model cost 0.658 seconds.
Prefix dict has been built succesfully.


In [8]:
df.content_cutted.head()

0    大 数据 产业 发展 受到 国家 重视 ， 而 大 数据 已经 上升 为 国家 战略 ， 未...
1    点击 上方 “ 硅谷 周边 ” 关注 我 ， 收到 最新 的 文章 哦 ！ 昨天 ， Goo...
2    国务院 总理 李克强 当地 时间 20 日 上午 在 纽约 下榻 饭店 同 美国 经济 、 ...
3    2016 年 ， 全峰 集团 持续 挖掘 大 数据 、 云 计算 、 “ 互联网 + ” 等...
4    贵州 理工学院 召开 大 数据分析 与 应用 专题 分享 会   借 “ 创响 中国 ” 贵...
Name: content_cutted, dtype: object

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [10]:
n_features = 1000

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                               max_features=n_features,
                               stop_words='english',
                               max_df = 0.5,
                               min_df = 10)
tf = tf_vectorizer.fit_transform(df.content_cutted)

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

In [12]:
n_topics = 5

lda = LatentDirichletAllocation(n_topics, max_iter=50,
                               learning_method='online',
                               learning_offset=50,
                               random_state=0)

In [13]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=5, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [14]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print('Topic #%d:' % topic_idx)
        print(''.join([feature_names[i]
                      for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [15]:
n_top_words = 20

In [16]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
学习模型使用算法方法机器可视化神经网络特征处理不同计算用户数据库系统如果分类训练一种基于
Topic #1:
这个就是可能没有如果他们自己很多什么不是但是或者因为时候这样现在电子一些所以孩子
Topic #2:
企业平台服务管理互联网数据分析公司产品用户业务行业客户金融创新实现价值系统能力工作需求
Topic #3:
中国2016市场增长10城市用户2015关注行业其中30人口检索阅读大众投资全国美国20
Topic #4:
人工智能学习领域智能机器人机器人类公司深度研究未来识别已经系统计算机目前医疗语音方面服务
()


In [18]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf ,tf_vectorizer)

D:\anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      25.852071        1       1  0.149635  0.021451
2      25.798362        1       2 -0.100942  0.102186
1      21.632325        1       3  0.127037 -0.112524
4      14.625269        1       4 -0.005510  0.118440
3      12.091972        1       5 -0.170219 -0.129553, topic_info=     Category         Freq  Term        Total  loglift  logprob
term                                                           
122   Default  2771.000000  人工智能  2771.000000  30.0000  30.0000
364   Default  4194.000000    学习  4194.000000  29.0000  29.0000
148   Default  3220.000000    企业  3220.000000  28.0000  28.0000
615   Default  2178.000000    机器  2178.000000  27.0000  27.0000
586   Default  1450.000000    智能  1450.000000  26.0000  26.0000
616   Default  1109.000000   机器人  1109.000000  25.0000  25.0000
91    Default  1773.000000    中国  1773.000000  24.0000  24.0000
992   Default  2003.000000    领域  2003.000000  23.0000  23.0000
125   Default  1017.000000    人类  1017.000000  22.0000  22.0000
929   Default  2265.000000    这个  2265.000000  21.0000  21.0000
645   Default  1765.000000    模型  1765.000000  20.0000  20.0000
778   Default  1839.000000    算法  1839.000000  19.0000  19.0000
18    Default  1004.000000  2016  1004.000000  18.0000  18.0000
193   Default  2440.000000    公司  2440.000000  17.0000  17.0000
779   Default  1362.000000    管理  1362.000000  16.0000  16.0000
401   Default  1798.000000    就是  1798.000000  15.0000  15.0000
678   Default  1336.000000    深度  1336.000000  14.0000  14.0000
417   Default  1411.000000    市场  1411.000000  13.0000  13.0000
340   Default   814.000000    增长   814.000000  12.0000  12.0000
607   Default  1748.000000    服务  1748.000000  11.0000  11.0000
331   Default   798.000000    城市   798.000000  10.0000  10.0000
424   Default  1734.000000    平台  1734.000000   9.0000   9.0000
129   Default  1067.000000    什么  1067.000000   8.0000   8.0000
1     Default   979.000000    10   979.000000   7.0000   7.0000
85    Default  1121.000000    业务  1121.000000   6.0000   6.0000
360   Default  1663.000000    如果  1663.000000   5.0000   5.0000
454   Default  1239.000000    很多  1239.000000   4.0000   4.0000
845   Default  1589.000000    行业  1589.000000   3.0000   3.0000
368   Default   626.000000    孩子   626.000000   2.0000   2.0000
718   Default   727.000000    电子   727.000000   1.0000   1.0000
...       ...          ...   ...          ...      ...      ...
187    Topic5   302.316730    全国   390.684292   1.8562  -5.1199
609    Topic5   136.545315    期间   177.943139   1.8478  -5.9147
8      Topic5   242.205138    15   317.223026   1.8428  -5.3416
716    Topic5   198.114045    申请   259.964435   1.8409  -5.5425
21     Topic5   163.196166    25   215.494631   1.8346  -5.7364
326    Topic5   236.632356    地区   313.108847   1.8326  -5.3648
340    Topic5   610.327748    增长   814.615688   1.8239  -4.4174
18     Topic5   750.205913  2016  1004.067948   1.8212  -4.2110
22     Topic5   326.256365    30   448.508389   1.7944  -5.0437
17     Topic5   485.246729  2015   708.467452   1.7342  -4.6467
331    Topic5   534.501579    城市   798.602410   1.7111  -4.5500
91     Topic5  1047.305694    中国  1773.569103   1.5859  -3.8774
1      Topic5   591.436906    10   979.981143   1.6077  -4.4488
4      Topic5   255.351191    11   352.936029   1.7890  -5.2887
417    Topic5   640.828649    市场  1411.161873   1.3232  -4.3686
250    Topic5   248.074821    北京   366.565127   1.7222  -5.3176
969    Topic5   311.601217    阅读   562.099252   1.5227  -5.0896
228    Topic5   238.056304    分别   349.847809   1.7276  -5.3588
13     Topic5   296.282344    20   546.958099   1.4996  -5.1400
198    Topic5   382.407822    关注   931.671734   1.2221  -4.8849
906    Topic5   272.897849    超过   500.115946   1.5069  -5.2222
713    Topic5   523.213916    用户  2486.159933   0.5541  -4.5714
203    Topic5   330.011311    其中   825.447658   1.1958  -5.

In [21]:
data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.show(data)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [08/Jan/2019 13:45:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2019 13:45:52] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2019 13:45:52] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2019 13:45:52] "GET /LDAvis.js HTTP/1.1" 200 -



stopping Server...
